In [28]:
!pip install tensorflow flask colabcode pyngrok

In [29]:
!ngrok authtoken #seu token privado

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [23]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [37]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.utils import to_categorical
from flask import Flask, request, jsonify
from tensorflow.keras.models import load_model
import numpy as np
from PIL import Image
import io
from flask_ngrok import run_with_ngrok
from pyngrok import ngrok
from flask import Flask, request, jsonify
import threading
import requests

In [3]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=x_train.shape[1:]))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=64, epochs=10, validation_data=(x_test, y_test), shuffle=True)

model.save('cifar10_model.h5')

170498071/170498071 [==============================] - 2s 0us/step
Epoch 1/10
782/782 [==============================] - 210s 266ms/step - loss: 1.3943 - accuracy: 0.4994 - val_loss: 1.1346 - val_accuracy: 0.5991
Epoch 2/10
782/782 [==============================] - 196s 251ms/step - loss: 0.9618 - accuracy: 0.6621 - val_loss: 0.9405 - val_accuracy: 0.6699
Epoch 3/10
782/782 [==============================] - 204s 260ms/step - loss: 0.7455 - accuracy: 0.7387 - val_loss: 0.7879 - val_accuracy: 0.7253
Epoch 4/10
782/782 [==============================] - 201s 257ms/step - loss: 0.5959 - accuracy: 0.7915 - val_loss: 0.7466 - val_accuracy: 0.7493
Epoch 5/10
782/782 [==============================] - 198s 254ms/step - loss: 0.4794 - accuracy: 0.8315 - val_loss: 0.7532 - val_accuracy: 0.7516
Epoch 6/10
782/782 [==============================] - 210s 268ms/step - loss: 0.3456 - accuracy: 0.8795 - val_loss: 0.8240 - val_accuracy: 0.7560
Epoch 7/10
782/782 [==============================] - 200

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [44]:
app = Flask(__name__)

model = load_model('cifar10_model.h5')

classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

def prepare_image(image):
    image = image.resize((32, 32))
    image = np.array(image)
    image = image.astype('float32') / 255.0
    image = np.expand_dims(image, axis=0)
    return image

@app.route('/predict', methods=['POST'])
def predict():
    if 'file' not in request.files:
        return jsonify({'error': 'No file provided'}), 400

    file = request.files['file']

    try:
        image = Image.open(io.BytesIO(file.read()))
        image = prepare_image(image)
        preds = model.predict(image)
        pred_class = classes[np.argmax(preds[0])]
        return jsonify({'category': pred_class}), 200
    except Exception as e:
        return jsonify({'error': str(e)}), 500

In [45]:
def run_flask():
    app.run(host='0.0.0.0', port=8000)

flask_thread = threading.Thread(target=run_flask)
flask_thread.start()

public_url = ngrok.connect(8000, bind_tls=True)
print(f'Public URL: {public_url}')

 * Serving Flask app '__main__'
 * Debug mode: off
Public URL: NgrokTunnel: "https://7033-34-31-240-5.ngrok-free.app" -> "http://localhost:8000"


Address already in use
Port 8000 is in use by another program. Either identify and stop that program, or start the server with a different port.


In [50]:
url = 'https://f8eb-34-31-240-5.ngrok-free.app/predict'

image_path = '/content/drive/MyDrive/imagens/11/barco.jpg'


with open(image_path, 'rb') as img:
    files = {'file': img}
    response = requests.post(url, files=files)

print(response.json())

1/1 [==============================] - 0s 38ms/step


INFO:werkzeug:127.0.0.1 - - [25/May/2024 01:05:06] "POST /predict HTTP/1.1" 200 -


{'category': 'ship'}


In [51]:
url = 'https://f8eb-34-31-240-5.ngrok-free.app/predict'

image_path = '/content/drive/MyDrive/imagens/11/aviao.jpg'


with open(image_path, 'rb') as img:
    files = {'file': img}
    response = requests.post(url, files=files)

print(response.json())

1/1 [==============================] - 0s 40ms/step


INFO:werkzeug:127.0.0.1 - - [25/May/2024 01:05:22] "POST /predict HTTP/1.1" 200 -


{'category': 'airplane'}


In [52]:
url = 'https://f8eb-34-31-240-5.ngrok-free.app/predict'

image_path = '/content/drive/MyDrive/imagens/11/carro.jpg'


with open(image_path, 'rb') as img:
    files = {'file': img}
    response = requests.post(url, files=files)

print(response.json())

1/1 [==============================] - 0s 37ms/step


INFO:werkzeug:127.0.0.1 - - [25/May/2024 01:05:32] "POST /predict HTTP/1.1" 200 -


{'category': 'automobile'}
